In [2]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("StatsBase")
Pkg.add("StatsModels")
Pkg.add("Statistics")
Pkg.add("GLM")



    Updating registry at `C:\Users\Lenovo\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\Lenovo\.julia\environments\v1.10\Project.toml`
  [3eaba693] + StatsModels v0.7.4
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Lenovo\.julia\environments\v1.10\Manifest.t

In [3]:
using DataFrames
using Statistics
using StatsBase
using CSV
using StatsModels
using GLM

# 1. Loading and processing the data

In [8]:
# 1. Load the data, either from the url or from the path provided

path = "C:/Users/User/Documents/GitHub/CausalAI-Course-1/"
data = CSV.read("../../../../data/wage2015_subsample_inference.csv", DataFrame) 
select!(data, Not("rownames"))

Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


In [9]:
# 2. As in Group Assignment 1, generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

types = Dict(:occ2 => String, :ind2 => String)

extra_flexible_formula = @formula(lwage ~ 1 + sex + (exp1 + exp2 + exp3 + exp4) * (hsg + scl + clg + ad + so + we + ne + occ2 + ind2) + (hsg + scl + clg + ad) * (so + we + ne + occ2 + ind2) + (so + we + ne) * (occ2 + ind2) + occ2 * ind2)
X_extra_flexible = modelmatrix(extra_flexible_formula, data);

In [10]:
# 2.1. Generate the array for the outcome variable Y and normalize it
y = data[:, "lwage"];
# y = (y .- mean(y)) ./ std(y) 

5150-element Vector{Float64}:
 2.2633643798407643
 3.872802292274865
 2.403126322215923
 2.634927936273247
 3.361976668508874
 2.4622152385859297
 2.9565115604007097
 2.9565115604007097
 2.4849066497880004
 2.9565115604007097
 ⋮
 3.117779707996832
 2.822980167776187
 3.1796551117149194
 2.6280074934286737
 2.6925460145662448
 3.138833117194664
 3.649658740960655
 3.4955080611333966
 2.8511510447428834

In [ ]:
# 2.2. Generate the array for the predictors X (do not generate an intercept) and normalize its colums.}



In [ ]:
# 3. Split between training and testing samples. The testing sample should be 10% of the total.